------------------------------------------------------
# Simulate data for the sdf example
------------------------------------------------------

#### Date: June 2017


In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal, norm

import statsmodels as sm
import statsmodels.tsa.api as tsa
from statsmodels.tsa.base.datetools import dates_from_str
import statsmodels.formula.api as smf

/home/szokeb/anaconda3/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# Example (but real) data from the statsmodel database
macro_data = sm.datasets.macrodata.load_pandas().data

# prepare the dates index
dates = macro_data[['year', 'quarter']].astype(int).astype(str)
quarterly = dates["year"] + "Q" + dates["quarter"]
quarterly = dates_from_str(quarterly)

macro_data = macro_data[['realgdp', 'realcons', 'realinv']]
macro_data.index = pd.DatetimeIndex(quarterly)
data = np.log(macro_data).diff().dropna()


In [3]:
#=======================================
# When we use multiple series
#=======================================

data = data.iloc[:, 1:]

lag = 1
model = tsa.VAR(data)
results = model.fit(lag)

M = len(results.names)
L = results.k_ar
mu = results.intercept
A = results.coefs    

error = np.asarray(results.resid)
T = error.shape[0]
Sigma = (error.T @ error)/T

In [4]:
#=======================================
# When we use a single series
#=======================================

data = data.iloc[:, 1]

lag = 1
model = tsa.AR(data)
results = model.fit(lag)

M, L = 1, 1
mu, A = results.params 
mu, A = np.asarray([mu]), np.asarray([[[A]]])

error = np.asarray([results.resid]).T
T = error.shape[0]
Sigma = np.asarray([[(error.T @ error)/T]])

In [5]:
def stationary_dist(mu, A, Sigma):

    M, L = A.shape[2], A.shape[0] 
    K = M*L
    
    mu_comp = np.zeros((K, 1))
    mu_comp[:M, 0] = mu
    A_row = np.hstack([A[i, :, :] for i in range(L)])
    A_comp = np.vstack([A_row, 
                        np.hstack([np.eye(M*(L-1)), np.zeros((M*(L-1), M))])])
    Sigma_comp = np.zeros((M*L, M*L))
    Sigma_comp[:M, :M] = Sigma

    mu_stationary = np.linalg.solve(np.eye(K) - A_comp, mu_comp)
    Sigma_stationary = sp.linalg.solve_discrete_lyapunov(A_comp, Sigma_comp)

    return mu_stationary, Sigma_stationary


def true_model(N, mu, A, Sigma):
    '''Simulating the true model'''
    
    M, L = A.shape[2], A.shape[0] 
    K = M*L
    
    mu_stationary, Sigma_stationary = stationary_dist(mu, A, Sigma)
        
    initial_x = multivariate_normal(mu_stationary.squeeze(), Sigma_stationary).rvs()
    shocks = multivariate_normal(np.zeros(len(mu)), Sigma)
    error = shocks.rvs(N - L).reshape(M, N - L)
    
    X = np.zeros((M, N))
    X[:, :L] = initial_x.reshape(L, M).T
    for t in range(N - L):
        AX = np.zeros((M, 1))
        for lag in range(L):
            AX += A[lag, :, :] @ X[:, t + L - 1 - lag].reshape(M, 1)
        X[:, L + t] = (mu.reshape(M, 1) + AX + error[:, t].reshape(M, 1)).squeeze()
    
    return pd.DataFrame(data = X.T, index = data.index[-N:]), error

In [6]:
# Simulate a dataset for Y...
simul, error_y = true_model(T, mu, A, Sigma)
# ...and demean it
simul = simul.sub(simul.mean(), axis='columns')

# The first columns of the data.dat file should be Y and its lagged version 
Y_path = np.asarray(simul)
Y_data = np.hstack([np.asarray(simul.iloc[1:, :]), np.asarray(simul.iloc[:-1, :])])

----------------------------------
## Generate a latent path

In [7]:
rho = .7
sigma = .01

x_path = []

x0 = (sigma/np.sqrt(1.0 - rho*rho))*norm.rvs()
x_path.append(x0)

error_x = sigma*norm.rvs(size=200)

for t in range(200):
    x_path.append(rho*x_path[t] + error_x[t])
X_path = np.asarray(x_path).reshape(201, 1)

In [8]:
shocks = np.vstack([error_y, error_x.reshape(1, 200)])

## Generate artificial returns

Suppose that returns have the following form (see RMT4 section 14.11.1 on page 590) 

$$\log R_{j, t+1} \sim \mathcal{N}\left(\nu_t(j) - \frac{1}{2}\alpha_t(j)'\alpha_t(j), \ \ \alpha_t(j)'\alpha_t(j) \right)$$

where $\nu_t(j)$ is a function of $(Y_t, X_t)$ that makes $E_t(S_{t+1}R_{j, t+1})=1$ become satisfied and 

$$\alpha_t(j) = \alpha_0(j) + \alpha_y(j)Y_t + \alpha_x(j)X_t$$

where $\alpha_0(j)$ is an $K+1$-vector and both $\alpha_y(j)$ and $\alpha_x(j)$ have $K+1$ (number of shocks) rows.

Suppose that among the elements of $Y$ we have a proxy for risk-free rate $r_{t+1}$ which is $t$-measurable. Then
 * $\delta_0 = \delta_x = \delta_{y\neq r} = 0$ and $\delta_r=1$
 * $\lambda^r_0 = \lambda^r_x = \lambda^r_y = 0$



We want return paths that are generated by the (true) shocks of $(Y_t, X_t)$.

$$\nu_t(j) = r_t + \alpha_t(j)'\lambda_t$$


In [9]:
nu = np.asarray([[ 0.02 , 
                  -0.01 , 
                   0.04 , 
                   0.0  , 
                  -0.02 , 
                   0.02 , 
                   0.001, 
                   0.005]]).T

alpha = np.asarray([[ .09, .05],
                    [ .02, .0 ],
                    [-.3 , .05],
                    [ .06, .02],
                    [ .3 ,-.1 ],
                    [-.3 ,-.2 ],
                    [ .03, .02],
                    [-.02, .01]])

In [10]:
logR = nu - np.diag(alpha @ alpha.T).reshape(8, 1)/2 + alpha @ shocks

In [11]:
plt.plot(logR.T)

In [12]:
simul_data = np.hstack([Y_data, logR.T])

In [13]:
np.savetxt('data.dat', simul_data, fmt='% 2.10f', delimiter=' ')
np.savetxt('initial_particle.dat', X_path, fmt='% 2.10f', delimiter=' ')

-----------------------------------------
## True parameter values

In [14]:
A

array([[[ 0.14847485]]])

In [15]:
np.linalg.cholesky(Sigma)

array([[ 0.04604768]])